In [1]:
import os

import gpytorch
import math
import numpy as np
import pandas as pd
import torch

from scipy.stats import norm

# torch.float64 to run in double precision, torch.float32 for single
dtype = torch.float32

# 'cuda' for GPU, 'cpu' for CPU
device = 'cuda'

[pyKeOps]: Warning, no cuda detected. Switching to cpu only.


In [2]:
if not os.path.exists('gpy_data.pth'):
    raise RuntimeError("Run gpy_wenda_save_data.ipynb first!")
    
train_x, train_y, test_x, test_y = torch.load('gpy_data.pth')

train_x = train_x.to(device=device, dtype=dtype)
train_y = train_y.to(device=device, dtype=dtype)
test_x = test_x.to(device=device, dtype=dtype)
test_y = test_y.to(device=device, dtype=dtype)

train_x = train_x
test_x = test_x

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

torch.Size([1866, 12979]) torch.Size([1866]) torch.Size([1001, 12979]) torch.Size([1001])


In [7]:
import math
# Initialize model and likelihood

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.LinearKernel()
        
        # Initialize variance to 1/d so that inner products between data points are ~ 1.
        # Unscaled inner products in train_x are so large that we lose precision.
        self.covar_module.variance = 1. / train_x.size(-1)
  
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device=device, dtype=dtype)

model = ExactGPModel(train_x, train_y, likelihood).to(device=device, dtype=dtype)

In [8]:
from tqdm.notebook import tqdm
from LBFGS import FullBatchLBFGS

def train_model_bfgs(model, likelihood, x, y, learning_rate,
                training_iter=10):
    lbfgs = FullBatchLBFGS(model.parameters(), lr=learning_rate)
    
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    
    def closure():
        model.zero_grad()
        output = model(x)
        loss = -mll(output, y)
        return loss

    loss = closure()
    loss.backward()

    train_iter = tqdm(range(training_iter))
    for i in train_iter:
        options = {"closure": closure, "current_loss": loss, "max_ls": 10}
        loss, _, lr, _, F_eval, G_eval, _, fail = lbfgs.step(options)
        train_iter.set_postfix({'loss': loss.item(), 'fail': fail})
        
        if fail:
            print('Convergence reached!')
            break
    
    return model, likelihood

In [9]:
%%time

model.train()
likelihood.train()

with gpytorch.settings.max_cholesky_size(100000), gpytorch.settings.cholesky_jitter(1e-5):
    model, likelihood = train_model_bfgs(
        model, likelihood, train_x, train_y, learning_rate=1., training_iter=15
    )

model.eval()
likelihood.eval()

Convergence reached!

Wall time: 576 ms


GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [10]:
# GPy learned:
## noise = 3.277223830593155e-15
## linear variance = 4.5426621979927965e-05

print('likelihood noise', likelihood.noise)
print('linear kernel variance', model.covar_module.variance)

likelihood noise tensor([0.0006], device='cuda:0', grad_fn=<AddBackward0>)
linear kernel variance tensor([[4.4735e-05]], device='cuda:0', grad_fn=<SoftplusBackward>)


In [13]:
# Save state dict

gene_number = 2
feature_model_format = 'model_{0:05d}'
output_dir = os.path.join("gpytorch_fast_feature_models", feature_model_format.format(gene_number))
if os.path.exists(output_dir) is False:
    os.makedirs(output_dir)
torch.save(model.state_dict(),os.path.join(output_dir, "state_dict.pth"))

In [14]:
# Get confidence score based on CDF of true target value on GP model
def getConfidence(model, x, y):
    with gpytorch.settings.fast_pred_var():
        f_preds = likelihood(model(x))
    mu = f_preds.mean
    sigma_sq = f_preds.variance
    sigma_sq = torch.sqrt(sigma_sq)
    res_normed = (y - mu) / sigma_sq
    res_normed = res_normed.cpu().detach().numpy()
    confidences = (1 - abs(norm.cdf(res_normed) - norm.cdf(-res_normed)))
    mu = mu.cpu().detach().numpy()
    sigma_sq = sigma_sq.cpu().detach().numpy()
    return mu, sigma_sq ** 2, confidences

In [15]:
# Write out confidence scores and predicted means and variances on target data
mean, var, conf = getConfidence(model, test_x, test_y)
print(conf.mean())

0.570456957621451


In [17]:
conf_file = os.path.join(output_dir, "confidences.txt")
np.savetxt(conf_file, conf, fmt='%.10f')
mean_file = os.path.join(output_dir, "predicted_means.txt")
np.savetxt(mean_file, mean, fmt='%.5f')
var_file = os.path.join(output_dir, "predicted_variances.txt")
np.savetxt(var_file, var)

In [31]:
print('** GPyTorch (fast)')
print(f'NLL      {torch.distributions.Normal(torch.from_numpy(mean), torch.from_numpy(var).sqrt()).log_prob(test_y.cpu()).mean().item():.3f}')
print(f'RMSE      {(torch.from_numpy(mean) - test_y.cpu()).pow(2).mean().item():.3f}')
print(f'Avg conf. {conf.mean():.3f}')

** GPyTorch (fast)
NLL      -1.019
RMSE      0.419
Avg conf. 0.570
